In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import libraries

In [1]:
! pip install wikipedia
! pip install nltk
! pip install rake-nltk

In [2]:
import wikipedia
import random
import tqdm
import nltk
import re
import string
import threading
import multiprocessing
import os
import shutil
from nltk.corpus import stopwords

In [3]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Dropout, Dense
from keras.layers import LSTM, Bidirectional, BatchNormalization
from keras.layers import SpatialDropout1D, Flatten, Conv2D
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.regularizers import l2, l1_l2
import keras

2021-08-03 09:10:36.791513: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [4]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('seaborn')
plt.rc('font', size=15)          # controls default text sizes
plt.rc('axes', titlesize=15)     # fontsize of the axes title
plt.rc('axes', labelsize=15)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=15)    # fontsize of the tick labels
plt.rc('ytick', labelsize=15)    # fontsize of the tick labels

from collections import Counter

In [5]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
wikipedia.set_lang("ru")

## Getting dataset

In [ ]:
def wikiTexts(topic, linksNum:list = [0], first:bool=True) -> (list, list) :
    links = []
    try:
        page = wikipedia.page(topic)
    except:
        return [], []
    
    try:
      links = page.links
    except:
      return result, [topic]
    
    result = [page.content]
    if len(linksNum) == 0:
        return result, [topic]
    
    random.shuffle(links)

    iterate = links[:linksNum[0]]
    resultLinks = [*iterate]
    if first:
      iterate = tqdm.tqdm_notebook(iterate)
    
    for link in iterate:
        contents = wikiTexts(link, linksNum[1:], False)
        result += contents[0]
        resultLinks += contents[1]
    return result, resultLinks

In [ ]:
def wikiThread(returnDict, topic, linksNum:list = [0]):
  returnDict[topic] = wikiTexts(topic, linksNum)

In [ ]:
depthSearch = [150, 2] # 150 первых ссылок, по 2 вторичных ссылки из каждой первой ссылки
jobs = []

manager = multiprocessing.Manager()
returnDict = manager.dict()

tasks = [
         (returnDict, "животные", depthSearch),
         (returnDict, "музыка", depthSearch),
         (returnDict, "спорт", depthSearch),
         (returnDict, "литература", depthSearch),
]
for i in range(len(tasks)):
  out_list = list()
  process = multiprocessing.Process(target=wikiThread, 
                                    args=tasks[i])
  jobs.append(process)

for j in jobs:
  j.start()

for j in jobs:
  j.join()

allTexts = {}
allLinks = {}
for k, v in returnDict.items() :
  allTexts[k], allLinks[k] = v


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that c

In [ ]:
import json
save_path = '/content/drive/MyDrive/DATAS/AIIJC/'
with open(save_path + 'aiijc_comand_data.json', 'w') as f :
  json.dump({'texts' : allTexts, 'links' : allLinks}, f)

In [14]:
import json
save_path = '/content/drive/MyDrive/DATAS/AIIJC/'
with open(save_path + 'aiijc_comand_data.json', 'r') as f :
  data = json.load(f)
  allTexts = data['texts']
  allLinks = data['links']

In [15]:
allTexts.keys()

dict_keys(['животные', 'музыка', 'спорт', 'литература'])

In [16]:
categories = ("животные", "музыка", "спорт", "литература")

In [17]:
for k in categories:
  texts = allTexts[k]
  links = allLinks[k]
  print(k, ":", sum([len(text.split(' ')) for text in texts]), "слов")
  print(*links, sep=", ")

животные : 818098 слов
Волосатики, Национальная парламентская библиотека (Япония), Гады, Триасовое вымирание, Аристотель, Ракообразные, Эвгленозои, Savalia savaglia, Виноградная улитка, Гаптофитовые водоросли, Палеогеновый период, Членистоногие, Holozoa, Энциклопедия жизни, Конечности, Tubulinea, Протерозой, Беспозвоночные, Telonemia, Malawimonada, Нервы, Биноминальная номенклатура, Криптофитовые водоросли, Food and Agriculture Organization, Швед, Doi, Криптисты, Тигр, Приматы, Sepiola atlantica, Земноводные, Tridacna squamosa, Breviatea, Heterolobosea, Зоология, Каменноугольный период, Medusozoa, Бесщупальцевые, Island Press, Видовой эпитет, Сперматозоид, Антропология, Зародышевые листки, Стрекающие, Животные-людоеды, Эволюция, Onychophora, Staurozoa, Анус, Страменопилы, Gemeinsame Normdatei, Коловратки, Меловой период, Палеозой, Гребневики, Головохоботные, Hoilungia hongkongensis, Jakobida, Форониды, Фламинго, Кожа, Amorphea, Юрский период, Хоаноциты, Паукообразные, Гнатостомулиды, М

In [18]:
stopWordsRu = set(stopwords.words("russian"))
punctuation = set(string.punctuation + "—")

def validateWord(word):
  if word in stopWordsRu or word in punctuation:
    return False
  allSymValid = False

  for sym in word:
    if not sym in punctuation:
      allSymValid = True
      break

  return allSymValid

def prepareText(text:str) -> list:
  out = []
  for word in nltk.word_tokenize(text):
    word = word.strip()
    if not validateWord(word):
      continue
    out.append(word.lower())
  return out


def generateTexts(texts, windowLen, need_prepare = True) -> list:
  out = []
  for text in tqdm.tqdm_notebook(texts):
    byWords = text.split(' ')
    for window in range(len(byWords) // windowLen):
      textPrepare = " ".join(byWords[window * windowLen: (window + 1) * windowLen])
      if need_prepare :
        prepared = prepareText(textPrepare)
      else :
        prepared = textPrepare
        
      if windowLen - len(prepared) > 0:
        prepared += [" "] * ( windowLen - len(prepared))
      out.append(prepared)
  return out

In [19]:
sentMaxLen = 300

In [20]:
prepared = {}
for cat in categories :
  prepared[cat] = generateTexts(allTexts[cat], sentMaxLen, False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/444 [00:00<?, ?it/s]

  0%|          | 0/431 [00:00<?, ?it/s]

  0%|          | 0/424 [00:00<?, ?it/s]

  0%|          | 0/433 [00:00<?, ?it/s]

In [21]:
dataFullX = []
dataFullY = []

for i, cat in enumerate(categories) :
  dataFullX += prepared[cat]
  dataFullY += len(prepared[cat]) * [[cat == x for x in categories]]

dataFullY = np.array(dataFullY)

In [22]:
len(dataFullX), len(dataFullY)

(11458, 11458)

In [23]:
maxWords = 15000
tokenizer = Tokenizer(num_words=maxWords, filters=string.punctuation + '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)

In [24]:
dataFullX[0], dataFullY[0]

('Живо́тные (лат. Animalia) — традиционно (со времён Аристотеля) выделяемая категория организмов, в настоящее время рассматриваемая в качестве биологического царства. Животные являются основным объектом изучения зоологии.\nЖивотные относятся к эукариотам (в клетках имеются ядра). Классическими признаками животных считаются: гетеротрофность (питание готовыми органическими соединениями) и способность активно передвигаться. Впрочем, существует немало животных, ведущих неподвижный образ жизни, а гетеротрофность свойственна грибам и некоторым растениям-паразитам.\nРусское слово «животное» образовано от «живот», в прошлом означавшего «жизнь, имущество». В быту под терминами «дикие животные», «домашние животные» часто понимаются только млекопитающие или четвероногие наземные позвоночные (млекопитающие, пресмыкающиеся и земноводные). Однако в науке за термином «животные» закреплено более широкое значение, соответствующее латинскому Animalia (см. выше). В научном смысле к животным, помимо млеко

In [25]:
tokenizer.fit_on_texts(map(lambda x: " ".join(x), dataFullX))
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 1211 unique tokens.


In [26]:
X = tokenizer.texts_to_sequences(map(lambda x: " ".join(x), dataFullX))
X = pad_sequences(X, maxlen=sentMaxLen, padding='post', truncating='post')
X.shape

(11458, 300)

In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(X, dataFullY, test_size = 0.2, shuffle=True, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(9166, 300) (9166, 4)
(2292, 300) (2292, 4)


## LSTM model

In [22]:
embDim = 128

In [38]:
model = Sequential([
                    Embedding(maxWords, embDim, input_length=X.shape[1]),
                    SpatialDropout1D(0.2),
                    LSTM(64, kernel_regularizer=l1_l2(0.0005)),
                    Dropout(0.2),
                    Dense(Y_test.shape[1], activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

In [39]:
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 300, 128)          1920000   
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 300, 128)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 260       
Total params: 1,969,668
Trainable params: 1,969,668
Non-trainable params: 0
_________________________________________________________________


In [40]:
epochs = 100
batch_size = 512

In [41]:
def scheduler(epoch, lr):
  if epoch < 90:
    return lr
  else:
    return lr / 1.7
history = model.fit(X_train,
                    Y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split=0.1,
                    callbacks=[
                            keras.callbacks.LearningRateScheduler(scheduler)  
                    ])

Epoch 1/100
17/17 [==============================] - 4s 113ms/step - loss: 3.8920 - accuracy: 0.2966 - val_loss: 3.2594 - val_accuracy: 0.2672
Epoch 2/100
17/17 [==============================] - 1s 74ms/step - loss: 3.0818 - accuracy: 0.3131 - val_loss: 2.6260 - val_accuracy: 0.2704
Epoch 3/100
17/17 [==============================] - 1s 76ms/step - loss: 2.4995 - accuracy: 0.3166 - val_loss: 2.1799 - val_accuracy: 0.2737
Epoch 4/100
17/17 [==============================] - 1s 76ms/step - loss: 2.0905 - accuracy: 0.3077 - val_loss: 1.8869 - val_accuracy: 0.2628
Epoch 5/100
17/17 [==============================] - 1s 76ms/step - loss: 1.8260 - accuracy: 0.2791 - val_loss: 1.6765 - val_accuracy: 0.2737
Epoch 6/100
17/17 [==============================] - 1s 74ms/step - loss: 1.6365 - accuracy: 0.3075 - val_loss: 1.5448 - val_accuracy: 0.2704
Epoch 7/100
17/17 [==============================] - 1s 76ms/step - loss: 1.5158 - accuracy: 0.3125 - val_loss: 1.4604 - val_accuracy: 0.2748
Epoch

In [27]:
model.evaluate(X_test, Y_test)

72/72 [==============================] - 1s 8ms/step - loss: 1.3900 - accuracy: 0.2723


[1.3899788856506348, 0.27225130796432495]

In [28]:
predictions = model.predict(X_test)
predictions = np.argmax(predictions, 1)
y_val = np.argmax(Y_test, 1)

In [29]:
import torch
save_path = '/content/drive/MyDrive/DATAS/AIIJC'
torch.save(model.state_dict(), save_path + '/lstm_model.pt')

NameError: ignored

## Bert model

In [28]:
!pip install transformers -q
!pip install tensorboardx -q
!pip install simpletransformers -q

In [29]:
dataFullX = ['[CLS] ' + el for el in dataFullX]
dataFullY = np.argmax(dataFullY, axis = 1)
X_train, X_test, Y_train, Y_test = train_test_split(dataFullX, dataFullY, test_size = 0.2, shuffle=True, random_state=42)

In [30]:
import pandas as pd

train_df = pd.DataFrame({
    'text': X_train,
    'labels':Y_train
})

print(train_df.head())

eval_df = pd.DataFrame({
    'text': X_test,
    'labels': Y_test
})

print(eval_df.head())

                                                text  labels
0  [CLS] и названия вида (видового эпитета). До Л...       0
1  [CLS] Гера, поддавшись либо уговорам Артемиды,...       2
2  [CLS] Э́ссен (нем. Essen [ˈɛsən]) — город земе...       3
3  [CLS] трудовые операции или ритуальная пляска,...       3
4  [CLS] Эхо́ (др.-греч. Ἠχώ) — персонаж древнегр...       2
                                                text  labels
0  [CLS] говорит о триединстве декаданса, символи...       3
1  [CLS] нескольких делений ядра без деления клет...       0
2  [CLS] префиксов DOI, регистрации DOI для объек...       0
3  [CLS] местах. Современная программа взрослых ч...       2
4  [CLS] Дворя́нство — привилегированное сословие...       3


In [31]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
model_args = ClassificationArgs(learning_rate = 1e-5,
                                num_train_epochs = 3,
                                #adam_epsilon = 1e-8,
                                #warmup_proportion = 0.1,
                                #weight_decay = 0.01,
                                #load_best_model_at_end = True,
                                best_model_dir = save_path + '/bert',
                                #metric_for_best_model = 'matthews_correlation',
                                train_batch_size = 16,
                                manual_seed = 42
                                )

model = ClassificationModel('bert', 'bert-base-multilingual-cased', num_labels=4, args=model_args, use_cuda=True)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [32]:
train_args = {'max_seq_length' : 512,
              #'do_lower_case' : True,
              'use_multiprocessing' : True,
              'train_batch_size' : 8,
              #'save_steps' : 50000,
              'eval_bacth_size' : 16,
              'overwrite_output_dir': True,
              # 'warmup_proportion' : 0.1,
              # 'weight_decay' : 0.01,
              'load_best_model_at_end' : True,
              'metric_for_best_model' : 'matthews_correlation',
              }
model.train_model(train_df = train_df, eval_df = eval_df, output_dir='outputs', args = train_args)

#result, model_outputs, wrong_predictions = model.eval_model(eval_df)

  0%|          | 0/9166 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/1146 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:942: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  model.parameters(), args.max_grad_norm


Running Epoch 1 of 3:   0%|          | 0/1146 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/1146 [00:00<?, ?it/s]

(3438, 0.5775033597806778)

In [33]:
result, predictions, wrong_predictions = model.eval_model(eval_df)

  0%|          | 0/2292 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/287 [00:00<?, ?it/s]

In [34]:
predictions = resul
predictions = np.argmax(predictions, 1)
y_val = Y_test

NameError: ignored

## Evaluate model

In [ ]:
predictions[0], y_val[0]

In [ ]:
from sklearn.metrics import *
import seaborn as sns

print("Precision: {0:6.2f}".format(precision_score(y_val, predictions, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(y_val, predictions, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(y_val, predictions, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(y_val, predictions)))
print(classification_report(y_val, predictions))
labels = [0, 1]
sns.heatmap(data=confusion_matrix(y_val, predictions), annot=True, fmt="d", cbar=False, xticklabels=labels, yticklabels=labels)
plt.title("Confusion matrix")
plt.show()

## Make submission

In [ ]:
import pandas as pd

In [ ]:
testDf = pd.read_csv('test.csv')

In [ ]:
testDf.head(5)

,id,task
0,0,Вопрос 1 Денис готовится провести соревнования...
1,1,Реши задачу.\nСколько коробок корма для собак ...
2,2,"Если групп элементов несколько, то применяется..."
3,3,Вопрос1 ...
4,4,Для наглядной геометрической иллюстрации объём...


In [ ]:
cnt = [len(x.split()) for x in testDf['task'].values]
plt.hist(cnt)
plt.show()

In [ ]:
sum([x > 256 for x in cnt])

In [ ]:
Xtest = tokenizer.texts_to_sequences(map(lambda x: str(x['task']), testDf.iloc))
Xtest = pad_sequences(Xtest, maxlen=sentMaxLen)

In [ ]:
predictions = model.predict(Xtest)
predictions

array([[0.09735209, 0.43359113, 0.16346458, 0.3055922 ],
       [0.07233533, 0.48948306, 0.1018154 , 0.33636627],
       [0.1266391 , 0.41387922, 0.18310064, 0.27638113],
       ...,
       [0.06528526, 0.3648363 , 0.1388722 , 0.43100628],
       [0.22779113, 0.59625584, 0.09307627, 0.08287673],
       [0.06749827, 0.5344242 , 0.12004167, 0.27803588]], dtype=float32)

In [ ]:
predictions = list(map(lambda x: categories[int(x)], np.argmax(predictions, axis=-1)))
predictions[:5]

['музыка', 'музыка', 'музыка', 'музыка', 'музыка']

In [ ]:
from rake_nltk import Rake

def getKeyWords(textSeries: pd.Series, threshold=4, minLen=1, maxLen=2):
    r = Rake(
        stopwords=stopWordsRu,
        punctuations=punctuation,
        min_length=1, max_length=2
    )
    results = []
    for i in range(textSeries.shape[0]):
      r.extract_keywords_from_text(testDf['task'].iloc[i])
      rowResult = r.get_ranked_phrases_with_scores()
      selected = 1
      while selected < len(rowResult):
        if rowResult[selected][0] < threshold:
          break
        selected += 1
      results.append(rowResult[:selected])
    return list(map(lambda task: list(map(lambda keyword: keyword[1], task)), results))

In [ ]:
keyWords = getKeyWords(testDf['task'])

In [ ]:
keyWordsReady = list(map( lambda x: ";".join(x), keyWords))

In [ ]:
sample = pd.read_csv('sample_submission.csv')


In [ ]:
sample['keywords'] = keyWordsReady

In [ ]:
sample['category'] = predictions

In [ ]:
sample.to_csv('submit.csv', index=False)

In [ ]:
sample['category'].value_counts()

музыка        353
литература    101
спорт          46
животные       14
Name: category, dtype: int64

In [ ]:
sample

,id,category,keywords
0,0,музыка,настольному хоккею;вариантов выбора
1,1,музыка,собак поместится;реши задачу
2,2,музыка,течение года;сдавали зачет;реши задачу;равно и...
3,3,музыка,суммы затрат;продукты питания;конце месяца;дом...
4,4,музыка,этими объектами;увлекающихся футболом;либо пон...
...,...,...,...
509,509,музыка,следующие вопросы;проанализируй диаграмму
510,510,музыка,сборке тренажеров;выполнить задание;6 дней;4 д...
511,511,литература,7 дней;1 день
512,512,музыка,ухватил пернатого;улетающую птицу;спрыгнул вни...
